# House prediction project
### Import the dataset and fit the data into the model

In [50]:
import pickle
import json
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor

from utils.dataloader import DataLoader
from settings. constants import TRAIN_CSV


with open('settings/specifications.json') as f:
    specifications = json.load(f)

raw_train = pd.read_csv(TRAIN_CSV)
x_columns = specifications['description']['X']
y_column = specifications['description']['y']

x_raw = raw_train[x_columns]

loader = DataLoader()
loader.fit(x_raw)
X = loader.load_data()
y = raw_train[y_column]

model = GradientBoostingRegressor()
model.fit(X, y)
with open('models/GBR.pickle', 'wb')as f:
    pickle.dump(model, f)

### Check the score of the model

In [55]:
import pickle
import json
import pandas as pd

from utils.dataloader import DataLoader
from settings. constants import VAL_CSV


with open('settings/specifications.json') as f:
    specifications = json.load(f)

x_columns = specifications['description']['X']
y_column = specifications['description']['y']

raw_val = pd.read_csv(VAL_CSV)
x_raw = raw_val[x_columns]

loader = DataLoader()
loader.fit(x_raw)
X = loader.load_data()
y = raw_val[y_column]
loaded_model = pickle.load(open('models/GBR.pickle', 'rb'))
y_pred = pd.DataFrame(loaded_model.predict(X))
print(loaded_model.score(X, y))
tocsv = pd.DataFrame()
tocsv['Id'] = raw_val['Id']
tocsv['SalePrice'] = y_pred[0]
tocsv.to_csv('data/submission.csv', index=False)

-18.495378452082797


In [52]:
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(mean_absolute_percentage_error(y, y_pred))

32.36361594883216


In [53]:
import json
import requests
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from utils import DataLoader, Estimator
from settings. constants import TRAIN_CSV, VAL_CSV

with open('settings/specifications.json') as f:
    specifications = json.load(f)

info = specifications['description']
x_columns, y_column, metrics = info['X'], info['y'], info['metrics']

train_set = pd.read_csv(TRAIN_CSV, header=0)
val_set = pd.read_csv(VAL_CSV, header=0)

train_x, train_y = train_set[x_columns], train_set[y_column]
val_x, val_y = val_set[x_columns], val_set[y_column]

loader = DataLoader()
loader.fit(val_x)
val_processed = loader.load_data()
print('data: ', val_processed[:10])

req_data = {'data': json.dumps(val_x.to_dict())}
response = requests.get('http://0.0.0.0:8000/predict', data=req_data)
api_predict = response.json()['prediction']
print('predict: ', api_predict[:10])

api_score = eval(metrics)(val_y, api_predict)
print('accuracy: ', api_score)

data:     MSSubClass  MSZoning  LotFrontage  LotArea  Street  LotShape  LandContour  \
0          20         2         80.0    11622       1         3            3   
1          20         3         81.0    14267       1         0            3   
2          60         3         74.0    13830       1         0            3   
3          60         3         78.0     9978       1         0            3   
4         120         3         43.0     5005       1         0            1   
5          60         3         75.0    10000       1         0            3   
6          20         3         67.0     7980       1         0            3   
7          60         3         63.0     8402       1         0            3   
8          20         3         85.0    10176       1         3            3   
9          20         3         70.0     8400       1         3            3   

   LotConfig  LandSlope  Neighborhood  ...  OpenPorchSF  EnclosedPorch  \
0          4          0            12 